In [1]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
from gensim import corpora, models
import pandas as pd

# データの読み込み

In [3]:
import matplotlib.pyplot as plt

In [4]:
target_data_food = pd.read_csv("super_cat1_food_data_target_user10_2500.csv", index_col=0)

In [5]:
target_data_food

,receipt_num,receipt_buydate,G_gyoutai_name,cat1,cat4,id
Unnamed: 0,,,,,,
129,57214817.0,2014-08-27,スーパー,食料品,たばこ,28890.0
146,56230608.0,2014-08-03,スーパー,食料品,たばこ,24134.0
154,57161430.0,2014-08-26,スーパー,食料品,チ－ズ,17724.0
155,57014455.0,2014-08-20,スーパー,食料品,チ－ズ,17800.0
156,56241337.0,2014-08-02,スーパー,食料品,チ－ズ,19343.0
...,...,...,...,...,...,...
1198181,56397797.0,2014-08-05,スーパー,食料品,食パン,12933.0
1198183,56417023.0,2014-08-08,スーパー,食料品,菓子パン・調理パン,12498.0
1198184,56914048.0,2014-08-20,スーパー,食料品,菓子パン・調理パン,12498.0


# コーパス作成

In [6]:
#全体のコーパス作成
grouped_cat4_data = target_data_food.groupby('id')['cat4'].apply(lambda x: [product.strip() for product in x]).reset_index()
user_cat4 = grouped_cat4_data['cat4'].tolist()

In [8]:
#年月別のコーパス作成
target_data_food['receipt_buydate'] = pd.to_datetime(target_data_food['receipt_buydate'])
grouped_cat4_data_ym = target_data_food.groupby([target_data_food['receipt_buydate'].dt.to_period("M"), 'id'])['cat4'].apply(lambda x: [product.strip() for product in x]).reset_index()
user_cat4_ym = grouped_cat4_data_ym['cat4'].tolist()

In [9]:
grouped_cat4_data_ym

,receipt_buydate,id,cat4
0,2014-04,0.0,"[つゆ・煮物料理の素, 煎餅・あられ, 煎餅・あられ, その他酒類, ベ－コン, ベ－コン,..."
1,2014-04,2.0,"[ジャム・ママレ－ド, 食パン, 食パン, 食パン, 食パン, 菓子パン・調理パン, 菓子パ..."
2,2014-04,4.0,"[液体茶, マヨネ－ズ, ビ－ル, ビ－ル, 煎餅・あられ, チョコレ－ト, 菓子パン・調理..."
3,2014-04,5.0,"[キャンディ, 豆腐類, 生麺・ゆで麺, 和風食品, 漬物, 牛乳, 煎餅・あられ, カレ－..."
4,2014-04,6.0,"[カレ－, キャンディ, 調理用ス－プ, デザ－ト類, ﾒﾆｭ-専用料理の素, ヨ－グルト,..."
...,...,...,...
203265,2015-03,40338.0,"[菓子パン・調理パン, 菓子パン・調理パン, 菓子パン・調理パン, １００％ジュ－ス, 納豆..."
203266,2015-03,40343.0,"[食パン, 醤油, ス－プ類, 洋風食品, 菓子パン・調理パン, 豆腐類, 牛乳, スナック..."
203267,2015-03,40344.0,"[食パン, 食パン, 食パン, 食パン, 食パン, 食パン, チョコレ－ト, チョコレ－ト,..."
203268,2015-03,40349.0,"[ワイン, 漬物, チ－ズ, チ－ズ, 魚貝類缶詰, チ－ズ, ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-, ..."


In [10]:
grouped_cat4_data_ym.groupby(grouped_cat4_data_ym['receipt_buydate']).count()

,id,cat4
receipt_buydate,,
2014-04,16939,16939
2014-05,16953,16953
2014-06,16942,16942
2014-07,16953,16953
2014-08,16958,16958
2014-09,16948,16948
2014-10,16954,16954
2014-11,16927,16927
2014-12,16940,16940


In [11]:
# 月ごとに分割して辞書に保存
#grouped_cat4_data_ym['receipt_buydate'] = pd.to_datetime(grouped_cat4_data_ym['receipt_buydate'])
monthly_data = {month: data for month, data in grouped_cat4_data_ym.groupby(grouped_cat4_data_ym['receipt_buydate'])['cat4']}


In [12]:
for month, data in monthly_data.items():
    print(month)

2014-04
2014-05
2014-06
2014-07
2014-08
2014-09
2014-10
2014-11
2014-12
2015-01
2015-02
2015-03


In [13]:
monthly_data[pd.Period('2014-04', freq='M')]

0        [つゆ・煮物料理の素, 煎餅・あられ, 煎餅・あられ, その他酒類, ベ－コン, ベ－コン,...
1        [ジャム・ママレ－ド, 食パン, 食パン, 食パン, 食パン, 菓子パン・調理パン, 菓子パ...
2        [液体茶, マヨネ－ズ, ビ－ル, ビ－ル, 煎餅・あられ, チョコレ－ト, 菓子パン・調理...
3        [キャンディ, 豆腐類, 生麺・ゆで麺, 和風食品, 漬物, 牛乳, 煎餅・あられ, カレ－...
4        [カレ－, キャンディ, 調理用ス－プ, デザ－ト類, ﾒﾆｭ-専用料理の素, ヨ－グルト,...
                               ...                        
16934    [畜肉ソ－セ－ジ, 醤油, 畜肉ソ－セ－ジ, 洋風食品, 畜肉ハム, 豆腐類, 豆腐類, ス...
16935    [ヨ－グルト, カレ－, 炭酸飲料, 菓子パン・調理パン, 菓子パン・調理パン, 生麺・ゆで...
16936    [ヨ－グルト, ヨ－グルト, 乾麺, 乾麺, 乾麺, 袋インスタント麺, 納豆, 煎餅・あら...
16937    [チョコレ－ト, その他スプレッド類, お茶漬けの素, チ－ズ, 畜肉ハム, チョコレ－ト,...
16938    [ヨ－グルト, チョコレ－ト, つゆ・煮物料理の素, つゆ・煮物料理の素, ｶｯﾌﾟｲﾝｽﾀ...
Name: cat4, Length: 16939, dtype: object

# 辞書の作成

In [14]:
texts = target_data_food['cat4'].tolist()
processed_docs = [[str(doc).lower().strip()] for doc in texts]
dictionary = corpora.Dictionary(processed_docs)

In [15]:
for doc in dictionary.values():
    print(doc)

たばこ
チ－ズ
ﾒﾆｭ-専用料理の素
その他スプレッド類
プレミックス
畜肉缶詰
シロップ類
ミネラルウォ－タ類
チョコレ－ト
キャンディ
スナック
野菜ジユ－ス
紅茶
ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-
香辛料
畜肉ソ－セ－ジ
その他酒類
ウィスキ－
ワイン
サラダ油・天ぷら油
果汁飲料
その他汎用調味料
砂糖
ジャム・ママレ－ド
その他飲料
蜂蜜
スポ－ツドリンク
ビ－ル
アイスクリ－ム
デザ－ト類
チュ－インガム
コ－ヒ－ドリンク
紅茶ドリンク
液体茶
マヨネ－ズ
炭酸飲料
冷凍調理
米飯類
食パン
揚げ物
乳酸飲料
料理酒
菓子パン・調理パン
牛乳
焼酎
はんぺん
かまぼこ
ちくわ
サイダ－
生麺・ゆで麺
豆腐類
袋インスタント麺
もずく・めかぶ
冷凍農産
漬物
洋風食品
醤油
介護食品
ス－プ類
ソ－ス
野菜缶詰
マ－ガリン類
ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺
フル－ツ缶詰
煎餅・あられ
ヨ－グルト
魚貝類缶詰
コ－ラ
シリアル類
つまみ類
つゆ・煮物料理の素
焼肉ｼｬﾌﾞｼｬﾌﾞのﾀﾚ
ﾎｲｯﾌﾟｸﾘ-ﾑ
スパゲッティ－
まぜご飯の素
１００％ジュ－ス
納豆
日本茶
食酢
わかめ・こんぶ類
マカロニ類
パスタソ－ス
ベ－コン
乾麺
焼豚
小麦粉
海苔
味噌
ベビ－フ－ド
みりん風調味料
その他合わせ調味料
佃煮
ｲﾝｽﾀﾝﾄｺ-ﾋ-
エッセンス類
ゴマ油
バタ－
ココア
畜肉ハム
味噌汁・吸物類
シチュ－ベ－ス
中華風食品
削り節
乳酸菌飲料
ﾌﾚｯｼｭｸﾘ-ﾑ
ふりかけ
栄養ドリンク
カレ－
豆乳
麦茶
ドレッシング
こうや豆腐
ぽん酢
パウチ入り食材
ﾚｷﾞｭﾗ-ｺ-ﾋ-
煮豆
ケチャップ
トマトジュ－ス
日本酒
唐揚げ粉
風味調味料
その他茶
和風食品
春雨・くず切り
米
その他練り製品
その他菓子
魚肉ソ－セ－ジ
てんぷら粉
低カロリ－甘味料
その他冷凍食品
食用塩
鍋補完材
シチュ－
美容・健康ドリンク
その他調理食品
パン粉
キャラメル
その他缶詰
その他麺類
栄養バランス食品
ｲﾝｽﾀﾝﾄ ｸﾘ-ﾑ
お茶漬けの素
調理用ス－プ
中国茶
玩具メ－カ－菓子
魚肉ハム
本みりん
煮干し
冷凍水産
その他食用油
その他食品
海藻サラダ
うまみ調味料
フリ－マルチ
複合調味料
育児用粉ミルク
液体だし
麦芽飲料
練りミルク
スキムミルク


In [16]:
len(pd.DataFrame(dictionary.values())[0].unique())

160

# コーパス関数

In [17]:
#コーパスを作成する関数
def create_corpus(m_data):
    corpus = [dictionary.doc2bow(doc) for doc in m_data]
    return corpus


In [18]:
#月別のデータのコーパス作成

#月別のコーパスを保存する辞書
corpus_dict = {}

for month, data in monthly_data.items():
    print(f'{month}のデータ完了')
    print()
    corpus_dict[month] = data


2014-04のデータ完了

2014-05のデータ完了

2014-06のデータ完了

2014-07のデータ完了

2014-08のデータ完了

2014-09のデータ完了

2014-10のデータ完了

2014-11のデータ完了

2014-12のデータ完了

2015-01のデータ完了

2015-02のデータ完了

2015-03のデータ完了



In [19]:
#corpus_dict

# ldaの実装

In [20]:
from gensim.models import LdaModel

In [21]:
#ldaモデルの作成
for month, corpus_org in corpus_dict.items():
    corpus = create_corpus(corpus_org)
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
    print(month)
    for topic_num , term in lda_model.print_topics():
        top_terms = term.split(" + ")
        top_5_terms = " + ".join(top_terms[0:5])
        print(f"({topic_num}, {top_5_terms})")
    print()

2014-04
(0, 0.148*"食パン" + 0.087*"菓子パン・調理パン" + 0.075*"牛乳" + 0.074*"豆腐類" + 0.044*"生麺・ゆで麺")
(1, 0.359*"菓子パン・調理パン" + 0.108*"冷凍調理" + 0.050*"食パン" + 0.032*"牛乳" + 0.032*"ヨ－グルト")
(2, 0.153*"スナック" + 0.072*"チョコレ－ト" + 0.067*"ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-" + 0.056*"キャンディ" + 0.047*"煎餅・あられ")
(3, 0.220*"アイスクリ－ム" + 0.079*"菓子パン・調理パン" + 0.070*"炭酸飲料" + 0.044*"デザ－ト類" + 0.037*"スナック")
(4, 0.178*"菓子パン・調理パン" + 0.078*"コ－ヒ－ドリンク" + 0.063*"ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺" + 0.053*"牛乳" + 0.046*"ヨ－グルト")
(5, 0.116*"ヨ－グルト" + 0.078*"豆腐類" + 0.047*"納豆" + 0.043*"牛乳" + 0.033*"生麺・ゆで麺")
(6, 0.239*"その他酒類" + 0.082*"ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺" + 0.039*"菓子パン・調理パン" + 0.034*"スナック" + 0.026*"ヨ－グルト")
(7, 0.164*"ビ－ル" + 0.046*"液体茶" + 0.034*"ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺" + 0.031*"冷凍調理" + 0.028*"牛乳")
(8, 0.097*"アイスクリ－ム" + 0.072*"牛乳" + 0.071*"チョコレ－ト" + 0.049*"豆腐類" + 0.038*"食パン")
(9, 0.193*"冷凍調理" + 0.044*"豆腐類" + 0.034*"牛乳" + 0.032*"畜肉ソ－セ－ジ" + 0.031*"ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺")

2014-05
(0, 0.327*"冷凍調理" + 0.063*"菓子パン・調理パン" + 0.031*"ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺" + 0.022*"ヨ－グルト" + 0.022*"食パン")
(1, 0.243*"その他酒類" + 0.057*"アイスクリ－ム" + 0.056*"ビ－

In [22]:
all_data = []

# 月ごとにLDAモデルからトピックを抽出し、リストに追加
for month, corpus_org in corpus_dict.items():
    corpus = create_corpus(corpus_org)
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
    for topic_num, term in lda_model.print_topics():
        top_terms = term.split(" + ")
        formatted_terms = ["{}({})".format(word.split("*")[1].replace('"', '').strip(), round(float(word.split("*")[0]), 3)) for word in top_terms[:5]]
        all_data.append([month, topic_num] + formatted_terms)


# データフレームの作成
columns = ['月', 'トピック', '1位', '2位', '3位', '4位', '5位']
df = pd.DataFrame(all_data, columns=columns)

# # CSVファイルへの書き出し
csv_file_path = 'lda_topic/Topics_Data_super.csv'
df.to_csv(csv_file_path, index=False)

In [ ]:
# LDAモデルの学習
# num_topics = 10
# lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=20, iterations=400)

# 各ドキュメントのトピック割合を取得
doc_topics = [lda_model.get_document_topics(doc) for doc in corpus]

# 各ドキュメントのトピック割合が最も高いトピックをそのドキュメントのトピックとして割り当て
doc_topic_labels = [max(topics, key=lambda x: x[1])[0] for topics in doc_topics]

# トピック割り当ての集計
topic_counts = pd.Series(doc_topic_labels).value_counts()

# 可視化
fig, ax = plt.subplots(figsize=(10, 6))
topic_counts.plot(kind='bar', ax=ax)
ax.set_xlabel('Topic')
ax.set_ylabel('Number of Documents')
ax.set_title(f'Document Topic Distribution (Total Topics: {num_topics})')

plt.tight_layout()
plt.show()

In [ ]:
lda_model